In [ ]:
# ======================================================
# CELL 1: CHECK GPU + HARDWARE SPECIFICATION
# ======================================================
import torch
import platform
import psutil

print("="*60)
print("GPU AVAILABILITY CHECK")
print("="*60)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
print("="*60)

print("\n" + "="*60)
print("HARDWARE SPECIFICATIONS")
print("="*60)
print(f"OS: {platform.system()} {platform.release()}")
print(f"CPU: {platform.processor()}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.2f} GB")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("GPU: Not available (using CPU)")
print("="*60)

In [ ]:
# ======================================================
# CELL 2: IMPORT LIBRARIES
# ======================================================
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import timm
import warnings
warnings.filterwarnings('ignore')

# Set seed untuk reproducibility
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
print("\n✅ Libraries imported")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# ======================================================
# CELL 3: CONFIGURATION
# ======================================================
# Path Configuration
DATA_DIR = "."
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
TEST_CSV = os.path.join(DATA_DIR, "test.csv")
TRAIN_IMG_DIR = os.path.join(DATA_DIR, "train")
TEST_IMG_DIR = os.path.join(DATA_DIR, "test")

# Training Configuration
CONFIG = {
    'img_size': 224,
    'batch_size': 8,
    'num_epochs': 10,
    'learning_rate': 3e-4,
    'weight_decay': 0.01,
    'num_workers': 0,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print("\n" + "="*60)
print("CONFIGURATION")
print("="*60)
for key, value in CONFIG.items():
    print(f"  {key}: {value}")
print("="*60)

In [ ]:
# ======================================================
# CELL 4: LOAD DATASET
# ======================================================
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

print("\n" + "="*60)
print("DATASET INFORMATION")
print("="*60)
print(f"Training samples: {len(train_df)}")
print(f"Test samples: {len(test_df)}")

# Clean data
train_df = train_df.dropna(subset=['label']).reset_index(drop=True)
test_df = test_df.dropna(subset=['label']).reset_index(drop=True)

# Class mapping
CLASS_NAMES = sorted(train_df['label'].unique())
class_to_idx = {cls: idx for idx, cls in enumerate(CLASS_NAMES)}
idx_to_class = {idx: cls for cls, idx in class_to_idx.items()}
num_classes = len(CLASS_NAMES)

print(f"Number of classes: {num_classes}")
print(f"Class names: {CLASS_NAMES}")

# Train-validation split
train_data, val_data = train_test_split(
    train_df, test_size=0.2, random_state=42, stratify=train_df['label']
)

print(f"\nData split:")
print(f"  Train: {len(train_data)}")
print(f"  Validation: {len(val_data)}")
print(f"  Test: {len(test_df)}")
print("="*60)

In [ ]:
# ======================================================
# CELL 5: DATA TRANSFORMS & DATASET CLASS
# ======================================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print("\n✅ Data transforms defined")

# Dataset Class
class FoodDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.class_to_idx = class_to_idx
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['filename']
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.class_to_idx[self.df.iloc[idx]['label']]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

print("✅ Dataset class defined")

In [ ]:
# ======================================================
# CELL 6: CREATE DATALOADERS
# ======================================================
train_dataset = FoodDataset(train_data, TRAIN_IMG_DIR, train_transform)
val_dataset = FoodDataset(val_data, TRAIN_IMG_DIR, val_transform)
test_dataset = FoodDataset(test_df, TEST_IMG_DIR, val_transform)

print(f"\n✅ Datasets created:")
print(f"  Train dataset: {len(train_dataset)} samples")
print(f"  Val dataset: {len(val_dataset)} samples")
print(f"  Test dataset: {len(test_dataset)} samples")

train_loader = DataLoader(
    train_dataset, 
    batch_size=CONFIG['batch_size'], 
    shuffle=True, 
    num_workers=CONFIG['num_workers'],
    drop_last=False
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=CONFIG['batch_size'], 
    shuffle=False, 
    num_workers=CONFIG['num_workers'],
    drop_last=False
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=CONFIG['batch_size'], 
    shuffle=False, 
    num_workers=CONFIG['num_workers'],
    drop_last=False
)

print(f"\n✅ DataLoaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

# Test one batch
try:
    test_iter = iter(test_loader)
    batch_images, batch_labels = next(test_iter)
    print(f"\n✅ Test batch verification:")
    print(f"  Batch images shape: {batch_images.shape}")
    print(f"  Batch labels shape: {batch_labels.shape}")
except Exception as e:
    print(f"\n❌ Error loading test batch: {e}")

In [ ]:
# ======================================================
# CELL 7: MODEL DEFINITIONS & UTILITY FUNCTIONS
# ======================================================
def get_vit_model(num_classes):
    return timm.create_model('vit_tiny_patch16_224', pretrained=True, num_classes=num_classes)

def get_deit_model(num_classes):
    return timm.create_model('deit_tiny_patch16_224', pretrained=True, num_classes=num_classes)

def get_swin_model(num_classes):
    return timm.create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=num_classes)

def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable = total - trainable
    return total, trainable, non_trainable

print("\n✅ Models defined: ViT, DeiT, Swin")
print("✅ Utility functions defined")

In [ ]:
# ======================================================
# CELL 8: TRAINING FUNCTIONS
# ======================================================
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(dataloader, desc='Training', leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()
    
    return running_loss / len(dataloader), 100 * correct / total


def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Validation', leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()
    
    return running_loss / len(dataloader), 100 * correct / total


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, model_name):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_acc = 0.0
    
    print(f"\n{'='*70}")
    print(f"TRAINING: {model_name}")
    print('='*70)
    print(f"{'Epoch':<8} {'Train Loss':<12} {'Train Acc':<12} {'Val Loss':<12} {'Val Acc':<12} {'Status':<10}")
    print('-'*70)
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Status indicator
        status = ""
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"best_{model_name}.pth")
            status = "💾 SAVED"
        
        print(f"{epoch+1:>3}/{num_epochs:<3} {train_loss:>11.4f} {train_acc:>10.2f}% {val_loss:>11.4f} {val_acc:>10.2f}% {status:<10}")
    
    print('-'*70)
    print(f"✅ Training completed! Best Val Acc: {best_val_acc:.2f}%")
    print('='*70)
    return history

print("✅ Training functions defined")

In [ ]:
# ======================================================
# CELL 9: EVALUATION & INFERENCE FUNCTIONS
# ======================================================
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    print(f"   Debug: Dataloader has {len(dataloader)} batches")
    
    try:
        with torch.no_grad():
            batch_count = 0
            for images, labels in dataloader:
                batch_count += 1
                images = images.to(device)
                labels = labels.to(device)
                
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                
                all_preds.extend(preds.cpu().numpy().tolist())
                all_labels.extend(labels.cpu().numpy().tolist())
            
            print(f"   Debug: Processed {batch_count} batches, collected {len(all_preds)} predictions")
        
        if len(all_preds) == 0 or len(all_labels) == 0:
            print(f"   ⚠️ Warning: No predictions collected!")
            return {
                'accuracy': 0.0,
                'precision': 0.0,
                'recall': 0.0,
                'f1_score': 0.0,
                'precision_per_class': np.zeros(num_classes),
                'recall_per_class': np.zeros(num_classes),
                'f1_per_class': np.zeros(num_classes),
                'confusion_matrix': np.zeros((num_classes, num_classes))
            }
        
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, all_preds, average='weighted', zero_division=0
        )
        precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(
            all_labels, all_preds, average=None, zero_division=0, labels=list(range(num_classes))
        )
        cm = confusion_matrix(all_labels, all_preds, labels=list(range(num_classes)))
        
        return {
            'accuracy': accuracy * 100,
            'precision': precision * 100,
            'recall': recall * 100,
            'f1_score': f1 * 100,
            'precision_per_class': precision_per_class * 100,
            'recall_per_class': recall_per_class * 100,
            'f1_per_class': f1_per_class * 100,
            'confusion_matrix': cm
        }
    
    except Exception as e:
        print(f"   ❌ Error during evaluation: {e}")
        import traceback
        traceback.print_exc()
        return {
            'accuracy': 0.0,
            'precision': 0.0,
            'recall': 0.0,
            'f1_score': 0.0,
            'precision_per_class': np.zeros(num_classes),
            'recall_per_class': np.zeros(num_classes),
            'f1_per_class': np.zeros(num_classes),
            'confusion_matrix': np.zeros((num_classes, num_classes))
        }


def measure_inference_time(model, dataloader, device, warmup=10):
    model.eval()
    times = []
    
    print(f"   Debug: Dataloader has {len(dataloader)} batches")
    
    try:
        with torch.no_grad():
            warmup_count = 0
            for images, _ in dataloader:
                if warmup_count >= warmup:
                    break
                images = images.to(device)
                _ = model(images)
                warmup_count += 1
            
            print(f"   Debug: Warmup completed with {warmup_count} batches")
            
            batch_count = 0
            for images, _ in dataloader:
                batch_count += 1
                images = images.to(device)
                
                if torch.cuda.is_available():
                    torch.cuda.synchronize()
                
                start_time = time.time()
                _ = model(images)
                
                if torch.cuda.is_available():
                    torch.cuda.synchronize()
                
                end_time = time.time()
                times.append(end_time - start_time)
            
            print(f"   Debug: Measured {batch_count} batches, collected {len(times)} times")
        
        if len(times) == 0:
            print(f"   ⚠️ Warning: No inference times collected!")
            return {
                'avg_per_image_ms': 0.0,
                'std_per_image_ms': 0.0,
                'total_test_time_s': 0.0,
                'throughput_img_per_s': 0.0
            }
        
        avg_batch_time = np.mean(times)
        std_batch_time = np.std(times)
        avg_per_image = avg_batch_time / CONFIG['batch_size']
        std_per_image = std_batch_time / CONFIG['batch_size']
        throughput = 1.0 / avg_per_image if avg_per_image > 0 else 0.0
        
        return {
            'avg_per_image_ms': avg_per_image * 1000,
            'std_per_image_ms': std_per_image * 1000,
            'total_test_time_s': sum(times),
            'throughput_img_per_s': throughput
        }
    
    except Exception as e:
        print(f"   ❌ Error during inference measurement: {e}")
        import traceback
        traceback.print_exc()
        return {
            'avg_per_image_ms': 0.0,
            'std_per_image_ms': 0.0,
            'total_test_time_s': 0.0,
            'throughput_img_per_s': 0.0
        }

print("✅ Evaluation and inference functions defined")

In [ ]:
# ======================================================
# CELL 10: TRAIN ALL MODELS
# ======================================================
device = CONFIG['device']
criterion = nn.CrossEntropyLoss()

results = {}
models_config = [
    ('ViT_Tiny', get_vit_model),
    ('DeiT_Tiny', get_deit_model),
    ('Swin_Tiny', get_swin_model)
]

print("\n" + "="*70)
print("STARTING TRAINING FOR ALL MODELS")
print("="*70)

for model_name, model_fn in models_config:
    print(f"\n{'█'*70}")
    print(f"MODEL: {model_name}")
    print('█'*70)
    
    # Initialize
    model = model_fn(num_classes).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], 
                           weight_decay=CONFIG['weight_decay'])
    
    # Count parameters
    total_params, trainable_params, non_trainable_params = count_parameters(model)
    model_size_mb = total_params * 4 / (1024 ** 2)
    
    print(f"\n📊 Model Information:")
    print(f"   Total Parameters    : {total_params:,}")
    print(f"   Trainable Parameters: {trainable_params:,}")
    print(f"   Model Size          : {model_size_mb:.2f} MB")
    
    # Train
    history = train_model(model, train_loader, val_loader, criterion, 
                         optimizer, CONFIG['num_epochs'], device, model_name)
    
    # Evaluate
    print(f"\n📈 Evaluating on Test Set...")
    try:
        model.load_state_dict(torch.load(f"best_{model_name}.pth", map_location=device))
        model.to(device)
        eval_results = evaluate_model(model, test_loader, device)
        
        print(f"\n🎯 Test Results:")
        print(f"   Accuracy : {eval_results['accuracy']:.2f}%")
        print(f"   Precision: {eval_results['precision']:.2f}%")
        print(f"   Recall   : {eval_results['recall']:.2f}%")
        print(f"   F1-Score : {eval_results['f1_score']:.2f}%")
    except Exception as e:
        print(f"   ❌ Error loading or evaluating model: {e}")
        import traceback
        traceback.print_exc()
        eval_results = {
            'accuracy': 0.0,
            'precision': 0.0,
            'recall': 0.0,
            'f1_score': 0.0,
            'precision_per_class': np.zeros(num_classes),
            'recall_per_class': np.zeros(num_classes),
            'f1_per_class': np.zeros(num_classes),
            'confusion_matrix': np.zeros((num_classes, num_classes))
        }
    
    # Measure inference
    print(f"\n⚡ Measuring Inference Time...")
    inference_time = measure_inference_time(model, test_loader, device)
    print(f"   Per Image : {inference_time['avg_per_image_ms']:.2f} ± {inference_time['std_per_image_ms']:.2f} ms")
    print(f"   Throughput: {inference_time['throughput_img_per_s']:.2f} images/sec")
    
    # Store results
    results[model_name] = {
        'parameters': {
            'total': total_params,
            'trainable': trainable_params,
            'non_trainable': non_trainable_params,
            'size_mb': model_size_mb
        },
        'performance': eval_results,
        'inference_time': inference_time,
        'history': history
    }
    
    del model, optimizer
    torch.cuda.empty_cache()
    
    print(f"\n✅ {model_name} completed!\n")

print("\n" + "="*70)
print("✅ ALL MODELS TRAINING COMPLETED")
print("="*70)

In [ ]:
# ======================================================
# CELL 11: COMPARISON TABLES
# ======================================================
print("\n" + "="*100)
print("MODEL COMPARISON TABLE")
print("="*100)
print(f"{'Model':<15} {'Params(M)':<12} {'Size(MB)':<10} {'Acc(%)':<10} {'Prec(%)':<10} {'Rec(%)':<10} {'F1(%)':<10} {'Inf(ms)':<10}")
print("="*100)

for model_name, result in results.items():
    print(f"{model_name:<15} "
          f"{result['parameters']['total']/1e6:<12.2f} "
          f"{result['parameters']['size_mb']:<10.2f} "
          f"{result['performance']['accuracy']:<10.2f} "
          f"{result['performance']['precision']:<10.2f} "
          f"{result['performance']['recall']:<10.2f} "
          f"{result['performance']['f1_score']:<10.2f} "
          f"{result['inference_time']['avg_per_image_ms']:<10.2f}")

print("="*100)

# Per-class metrics table
print("\n" + "="*80)
print("PER-CLASS PERFORMANCE METRICS")
print("="*80)

for model_name, result in results.items():
    print(f"\n{model_name}:")
    print(f"{'Class':<20} {'Precision(%)':<15} {'Recall(%)':<15} {'F1-Score(%)':<15}")
    print("-"*70)
    
    prec_per_class = result['performance']['precision_per_class']
    rec_per_class = result['performance']['recall_per_class']
    f1_per_class = result['performance']['f1_per_class']
    
    if len(prec_per_class) == len(CLASS_NAMES):
        for idx, class_name in enumerate(CLASS_NAMES):
            print(f"{class_name:<20} {prec_per_class[idx]:<15.2f} {rec_per_class[idx]:<15.2f} {f1_per_class[idx]:<15.2f}")
    else:
        print(f"⚠️ Warning: Per-class metrics length ({len(prec_per_class)}) doesn't match number of classes ({len(CLASS_NAMES)})")
        for idx in range(min(len(prec_per_class), len(CLASS_NAMES))):
            class_name = CLASS_NAMES[idx] if idx < len(CLASS_NAMES) else f"Class_{idx}"
            print(f"{class_name:<20} {prec_per_class[idx]:<15.2f} {rec_per_class[idx]:<15.2f} {f1_per_class[idx]:<15.2f}")
    
    print("-"*70)
    print(f"{'AVERAGE':<20} {result['performance']['precision']:<15.2f} {result['performance']['recall']:<15.2f} {result['performance']['f1_score']:<15.2f}")
    print()

print("="*80)

In [ ]:
# ======================================================
# CELL 12: TRAINING CURVES VISUALIZATION
# ======================================================
print("\n📊 Generating Training Curves...")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Loss
for model_name, result in results.items():
    epochs = range(1, len(result['history']['train_loss']) + 1)
    axes[0].plot(epochs, result['history']['train_loss'], '-o', label=f'{model_name} Train', linewidth=2, markersize=4)
    axes[0].plot(epochs, result['history']['val_loss'], '--s', label=f'{model_name} Val', linewidth=2, markersize=4)

axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=9)
axes[0].grid(True, alpha=0.3)

# Plot 2: Accuracy
for model_name, result in results.items():
    epochs = range(1, len(result['history']['train_acc']) + 1)
    axes[1].plot(epochs, result['history']['train_acc'], '-o', label=f'{model_name} Train', linewidth=2, markersize=4)
    axes[1].plot(epochs, result['history']['val_acc'], '--s', label=f'{model_name} Val', linewidth=2, markersize=4)

axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Training & Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Training curves displayed")

In [ ]:
# ======================================================
# CELL 13: CONFUSION MATRICES
# ======================================================
print("\n📊 Generating Confusion Matrices...")

n_models = len(results)
fig, axes = plt.subplots(1, n_models, figsize=(6*n_models, 5))
if n_models == 1:
    axes = [axes]

for idx, (model_name, result) in enumerate(results.items()):
    cm = result['performance']['confusion_matrix']
    
    # Plot confusion matrix
    im = axes[idx].imshow(cm, cmap='Blues', aspect='auto', interpolation='nearest')
    axes[idx].set_title(f'{model_name}\nConfusion Matrix', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Predicted', fontsize=12)
    axes[idx].set_ylabel('Actual', fontsize=12)
    
    # Colorbar
    cbar = plt.colorbar(im, ax=axes[idx])
    cbar.set_label('Count', fontsize=10)
    
    # Annotations
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            text_color = 'white' if cm[i, j] > cm.max()/2 else 'black'
            axes[idx].text(j, i, int(cm[i, j]), 
                          ha="center", va="center", 
                          color=text_color, fontsize=11, fontweight='bold')
    
    # Ticks
    axes[idx].set_xticks(range(len(CLASS_NAMES)))
    axes[idx].set_yticks(range(len(CLASS_NAMES)))
    axes[idx].set_xticklabels(CLASS_NAMES, rotation=45, ha='right', fontsize=10)
    axes[idx].set_yticklabels(CLASS_NAMES, fontsize=10)

plt.tight_layout()
plt.show()

print("✅ Confusion matrices displayed")

In [ ]:
# ======================================================
# CELL 14: PERFORMANCE METRICS COMPARISON
# ======================================================
print("\n📊 Generating Performance Comparison...")

fig, ax = plt.subplots(figsize=(12, 6))

metrics = ['accuracy', 'precision', 'recall', 'f1_score']
x = np.arange(len(metrics))
width = 0.25
model_names = list(results.keys())

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

# Print data for verification
print("Checking results data:")
for model_name in model_names:
    print(f"{model_name}:")
    for metric in metrics:
        value = results[model_name]['performance'][metric]
        print(f"  {metric}: {value:.2f}")

# Plot bars
for i, model_name in enumerate(model_names):
    values = [results[model_name]['performance'][m] for m in metrics]
    bars = ax.bar(x + i*width, values, width, label=model_name, color=colors[i], alpha=0.8)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height:.1f}',
                ha='center', va='bottom', fontsize=9, fontweight='bold')

ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
ax.set_ylabel('Score (%)', fontsize=12, fontweight='bold')
ax.set_title('Performance Metrics Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x + width)
ax.set_xticklabels(['Accuracy', 'Precision', 'Recall', 'F1-Score'], fontsize=11)
ax.legend(fontsize=10, loc='lower right')
ax.grid(True, axis='y', alpha=0.3)
ax.set_ylim([0, 110])

plt.tight_layout()
plt.show()

print("✅ Performance comparison displayed")

In [ ]:
# ======================================================
# CELL 15: MODEL RECOMMENDATIONS
# ======================================================
print("\n" + "="*80)
print("MODEL RECOMMENDATIONS")
print("="*80)

# Best accuracy
best_acc_model = max(results.items(), key=lambda x: x[1]['performance']['accuracy'])
print(f"\n✅ BEST ACCURACY:")
print(f"   Model: {best_acc_model[0]}")
print(f"   Accuracy: {best_acc_model[1]['performance']['accuracy']:.2f}%")
print(f"   Use case: When highest classification accuracy is critical")

# Best F1-Score
best_f1_model = max(results.items(), key=lambda x: x[1]['performance']['f1_score'])
print(f"\n🎯 BEST F1-SCORE:")
print(f"   Model: {best_f1_model[0]}")
print(f"   F1-Score: {best_f1_model[1]['performance']['f1_score']:.2f}%")
print(f"   Use case: When balanced precision-recall is needed")

# Fastest inference
best_speed_model = min(results.items(), key=lambda x: x[1]['inference_time']['avg_per_image_ms'])
print(f"\n⚡ FASTEST INFERENCE:")
print(f"   Model: {best_speed_model[0]}")
print(f"   Speed: {best_speed_model[1]['inference_time']['avg_per_image_ms']:.2f} ms/image")
print(f"   Throughput: {best_speed_model[1]['inference_time']['throughput_img_per_s']:.2f} images/sec")
print(f"   Use case: Real-time applications, edge devices")

# Most efficient (smallest parameters)
best_size_model = min(results.items(), key=lambda x: x[1]['parameters']['total'])
print(f"\n💾 SMALLEST MODEL:")
print(f"   Model: {best_size_model[0]}")
print(f"   Parameters: {best_size_model[1]['parameters']['total']/1e6:.2f}M")
print(f"   Size: {best_size_model[1]['parameters']['size_mb']:.2f} MB")
print(f"   Use case: Memory-constrained environments, mobile deployment")

# Best trade-off (accuracy vs speed)
print(f"\n⚖️ BEST TRADE-OFF (Accuracy vs Speed):")
tradeoff_scores = {}
for model_name, result in results.items():
    acc_score = result['performance']['accuracy'] / 100
    speed_score = 1 / result['inference_time']['avg_per_image_ms']
    tradeoff_scores[model_name] = 0.6 * acc_score + 0.4 * speed_score

best_tradeoff = max(tradeoff_scores.items(), key=lambda x: x[1])
print(f"   Model: {best_tradeoff[0]}")
print(f"   Accuracy: {results[best_tradeoff[0]]['performance']['accuracy']:.2f}%")
print(f"   Inference: {results[best_tradeoff[0]]['inference_time']['avg_per_image_ms']:.2f} ms/image")
print(f"   Use case: Production applications requiring both accuracy and efficiency")

print("\n" + "="*80)

In [ ]:
# ======================================================
# CELL 16: SUMMARY & CONCLUSION
# ======================================================
print("\n" + "="*60)
print("✅ EXPERIMENT COMPLETED")
print("="*60)
print("\nGenerated files:")
print("  - best_ViT_Tiny.pth")
print("  - best_DeiT_Tiny.pth")
print("  - best_Swin_Tiny.pth")
print("\nAll visualizations displayed above.")
print("\nHardware used:")
if torch.cuda.is_available():
    print(f"  - GPU: {torch.cuda.get_device_name(0)}")
else:
    print(f"  - CPU: {platform.processor()}")
print("\n" + "="*60)
print("EXPERIMENT SUMMARY:")
print("="*60)
print("\nModels Trained:")
for model_name in results.keys():
    print(f"  ✓ {model_name}")
print(f"\nTotal Models: {len(results)}")
print(f"Dataset Classes: {num_classes}")
print(f"Device Used: {device.upper()}")
print("\n" + "="*60)